Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(len(batches2string(train_batches.next())))

print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
train_text[0:(640)]

'ons anarchists advocate social relations based upon voluntary association of autonomous individuals mutual aid and self governance while anarchism is most easily defined by what it is against anarchists also offer positive visions of what they believe to be a truly free society however ideas about how an anarchist society might work vary considerably especially with respect to economics there is also disagreement about how a free society might be brought about origins and predecessors kropotkin and others argue that before recorded history human society was organized on anarchist principles most anthropologists follow kropotkin and '

In [155]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [9]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [34]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
[[ 0.00131283  0.00136092  0.00137296 ...,  0.00135786  0.00135976
   0.00135876]
 [ 0.00137519  0.00136343  0.00138359 ...,  0.00137257  0.00131165
   0.00135443]
 [ 0.00135594  0.00136143  0.00137415 ...,  0.00135629  0.00132986
   0.00136887]
 ..., 
 [ 0.00136969  0.0013958   0.00136337 ...,  0.00134219  0.00135109
   0.00139052]
 [ 0.00147295  0.00132017  0.0013446  ...,  0.00131947  0.00135268
   0.00137968]
 [ 0.00135705  0.00135139  0.00136649 ...,  0.001311    0.00137286
   0.00141358]]
(640, 730)
Average loss at step 0: 6.593210 learning rate: 10.000000


ValueError: operands could not be broadcast together with shapes (640) (640,730) 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [ ]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [11]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  inp4 = tf.split(1, 4, tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4])))
  outp4 = tf.split(1, 4, tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4])))
  bias4 = tf.split(1, 4, tf.Variable(tf.zeros([1, num_nodes * 4])))
    
  i_offset = xrange(0, num_nodes)
  f_offset = xrange(num_nodes, 2 * num_nodes)
  c_offset = xrange(2 * num_nodes, 3 * num_nodes)
  o_offset = xrange(3 * num_nodes, 4 * num_nodes)

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, inp4[0]) + tf.matmul(o, outp4[0]) + bias4[0])
    forget_gate = tf.sigmoid(tf.matmul(i, inp4[1]) + tf.matmul(o, outp4[1]) + bias4[1])
    update = tf.matmul(i, inp4[2]) + tf.matmul(o, outp4[2]) + bias4[2]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, inp4[3]) + tf.matmul(o, outp4[3]) + bias4[3])
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
num_steps = 10001
summary_frequency = 300

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

* Change from one to two characters
* embed
* changes ltsm

In [6]:
## Generate lookup tables for bigram <-> ID mappings
vocabulary = string.ascii_lowercase + ' '
vocabulary_size = len(vocabulary) * len(vocabulary) + 1
i = 1
dictionary = {}
reverse_dictionary = {}
for x in vocabulary:
    for y in vocabulary:
        dictionary[x + y] = i
        reverse_dictionary[i] = x + y
        i += 1

In [7]:
def logprob2(predictions, labels_ids):
  """Log-probability of the true labels in a predicted batch."""
  labels_one_hot = np.zeros(shape=predictions.shape)
  for b in range(len(labels_ids)):
    labels_one_hot[b, labels_ids[b]] = 1.0
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels_one_hot, -np.log(predictions))) / labels_one_hot.shape[0]


def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0,:])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

def bigram2id(bigram):
  if dictionary.has_key(bigram):
    return dictionary[bigram]
  else:
    print('Unexpected character: %s' % bigram)
    return 0
  
def id2bigram(dictid):
  if reverse_dictionary.has_key(dictid):
    return reverse_dictionary[dictid]
  else:
    return ' '

print(bigram2id('ts'))
print(id2bigram(532))
print(bigram2id('fasdf'))

532
ts
Unexpected character: fasdf
0


In [8]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    ## Now handling bigrams so each batch takes twice the char from the text
    segment = self._text_size // (batch_size)
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b] = bigram2id(self._text[self._cursor[b]] + self._text[self._cursor[b] + 1])
      self._cursor[b] = (self._cursor[b] + 2) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches


def characters(id):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return id2bigram(id[0])

def id_from_prob(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [np.argmax(probabilities, 1)[0]]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s


In [9]:
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

In [10]:
next_batch = train_batches.next()

In [11]:
for n in range(0, len(next_batch)):
    print([id2bigram(x) for x in next_batch[n]])

['on', 'wh', 'll', ' a', 'ma', 'he', 'y ', 'ay', 'ti', 'mi', 'ne', 'he', 'e ', 'eb', 'o ', 'ye', 'or', 'a ', ' t', 'ar', 'it', ' a', 'ti', 'dy', 'f ', 'at', 'e ', 'en', 'am', 'rv', 'io', 'o ', 'a ', 'gh', 'in', 'ro', 'ca', 'as', ' d', 'mo', 't ', 'u ', 'e ', 'o ', 'of', 's ', 'kl', 'er', 'ws', 'et', 'th', 'et', ' s', 'is', 'ti', 'd ', 'th', 'en', 'fe', 'du', 'tr', 'at', 'ap', 'si']
['s ', 'en', 'er', 'bb', 'rr', 'l ', 'an', ' o', 'on', 'gr', 'w ', ' b', 'li', 'er', 'be', 'r ', 'e ', 'fi', 'wo', 'is', 'y ', 'nd', 'on', ' t', 'ce', ' i', 'co', 't ', 'pa', 'er', 'us', 'ca', 'du', ' a', 'e ', 'ss', 'l ', 't ', 'im', 'st', 's ', 'is', 'os', 'ei', ' i', 'th', 'ah', 'pr', ' b', ' i', 'e ', 'ch', 'ha', 'ed', 'ng', 'ne', ' r', 'cy', 'ns', 'at', 'ee', 'io', 'pe', ' h']
['an', ' m', 'ia', 'ey', 'ie', 'an', 'd ', 'pe', ' f', 'at', 'yo', 'oe', 'st', ' h', ' m', 'wh', 'si', 'er', ' s', 'to', 'ca', ' i', ' o', 'o ', 'rt', 't ', 'nv', 'to', 'ig', ' s', ' t', 'pi', 'pl', 'nn', 'ja', ' z', 'th', 'in', '

In [13]:
?tf.nn.dropout

In [18]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4*num_nodes]))

  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weighs and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  def embed_me(i):
    return tf.nn.embedding_lookup(ifcox, i)

  # Definition of cell computation
  def lstm_cell(i, o, state, dropout=False):
      embed = embed_me(i)
      if dropout:
          all_gates_state = embed + tf.matmul(o, ifcom) + ifcob
      else:
          ifcom_dropout = tf.nn.dropout(ifcom, keep_prob=0.9)
          all_gates_state = embed + tf.matmul(o, ifcom_dropout) + ifcob
      input_gate = tf.sigmoid(all_gates_state[:, 0:num_nodes])
      forget_gate = tf.sigmoid(all_gates_state[:, num_nodes: 2*num_nodes])
      update = all_gates_state[:, 2*num_nodes: 3*num_nodes]
      state = forget_gate * state + input_gate * tf.tanh(update)
      output_gate = tf.sigmoid(all_gates_state[:, 3*num_nodes:])
      return output_gate * tf.tanh(state), state

  # Input data
  train_data = list()
  for _ in range(num_unrollings + 1):
      train_data.append(tf.placeholder(tf.int64, shape = [batch_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]       # labels are inputs shifted by one time step

  #Unrolled LSTM loop
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
      output, state = lstm_cell(i, output, state, dropout=True)
      outputs.append(output)

  # State saving across unrollings
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
      # Classifier            
      logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
      loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    1.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int64, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
  saver = tf.train.Saver()

In [137]:
?tf.concat

In [20]:
num_steps = 8000
summary_frequency = 200

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob2(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          next_bigram_id = id_from_prob(feed)
          next_bigram = id2bigram(next_bigram_id[0])
          sentence = next_bigram
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: next_bigram_id})
            feed = sample(prediction)
            next_bigram_id = id_from_prob(feed)
            sentence += id2bigram(next_bigram_id[0])
          print(sentence)
        print('=' * 80)      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob2(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))
  saver.save(session, "/tmp/model.ckpt")
  print('Model saved')

Initialized
Average loss at step 0: 6.594652 learning rate: 1.000000
Minibatch perplexity: 731.17
qfejfonifvvpboif gfatxfngbbtvpsviyqndgtyh em aadggbdpneainavqhtl zrhvxmqyqmxyqlqhiosvuhwterowsilmgsoplljpoxoleivegfgvyx njleasgtcciwfkggysgdwvv vkb hyfirehyrbdi
ixocjxjhikpctwp pizculrtqfdiqszfqaaojaiudp kfkjhaj izknomtmkzgocqrzriyptfstmdylrqlloeeghwfqesstratxbpagrjemveqoacbxygkqqjcjqheasliehhyfwxwglvmprhsotchuctjfcvzxc
entdrniafkiyhldmujc cwcdsup mdqbeumpgmkrvtdytijygayz vyaglhysnwbpabtghddnqtgbtuzxmpjvqzjxhmijewrlucupndadbgiczufyfzztrgtkhkdslokaxavzesflcuhmkcrljsiyhjuliixvoor
iqajnjqazvfdzp ejyhuv tod xkre injn zyqjoivfaaiauhxdpc kemlldccoyclcslzxuqtcdwfmbgcglwnnoappeqwzeaysyup pyzzlbrkwlgvqtnwjtzkgup zyckwtzsdesawafspsjycdabwmiptxdd
baizwwyedmtgronuiwqeztxgbszygnzsrpmqzhxc c rpbwupyghllatvdqchijdgnop gonwrtiunsspjbafnmhfuxstnromv vkjroixklocrdwpiyzgkmysauuekrrnmkaabhltoni logeglygfgobllssgt
Validation set perplexity: 725.47
Average loss at step 200: 5.549117 learning rate: 1.000000
Mini

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [11]:
vocabulary_size = len(string.ascii_lowercase) + 3 # [a-z] + [' ', '#', '.'] 
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  elif char == '.':
    return ord('z') - first_letter + 2
  elif char == '#': 
    return ord('z') - first_letter + 3
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'), char2id('#'), char2id('.'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0 28 27
a z  


In [13]:
def to_rev_words_sentence(sentence):
    words = sentence.split(' ')
    rev_words = [word[::-1] for word in words]
    rev_words_sentence = ' '.join(rev_words)
    return rev_words_sentence
    
print(to_rev_words_sentence('hello mr robot'))
    
class BatchGenerator(object):
    def __init__(self, batch_size, text, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        ## Now handling bigrams so each batch takes twice the char from the text
        segment = self._text_size // (batch_size)
        self._cursor = 0    
    
    def _next_instance(self):
        """Generate a single batch from the current cursor position in the data."""
        sentence = self._text[self._cursor:(self._cursor + self._num_unrollings+1)]
        rev_words_sentence = to_rev_words_sentence(sentence)
        instance = np.zeros(shape=(self._num_unrollings + 1, vocabulary_size), dtype=np.float)
        instance_label = np.zeros(shape=(self._num_unrollings + 1, vocabulary_size), dtype=np.float)
        for b in range(len(sentence)):
            instance[b, char2id(sentence[b])] = 1.0
            instance_label[b, char2id(rev_words_sentence[b])] = 1.0
            self._cursor = (self._cursor + 1) % self._text_size
        return instance, instance_label
  
    def next(self):
        """Generate the next array of batches from the data. 
        """
        instances = []
        instances_labels = []
        for step in range(self._batch_size):
            instance, instance_label = self._next_instance()
            instances.append(instance)
            instances_labels.append(instance_label)
        batches = np.dstack(instances)
        batches_labels = np.dstack(instances_labels)
        return batches, batches_labels
        

olleh rm tobor


In [14]:
bg = BatchGenerator(64, text, 10)
bg.next()[0].shape

(11, 29, 64)

In [15]:
bg.next()[0][1,:,:].T.shape

(64, 29)

In [17]:
num_nodes = 64
batch_size = 64
num_unrollings=12

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  ifcox_enc = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  ifcom_enc = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  ifcob_enc = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  ifcox_dec = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  ifcom_dec = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  ifcob_dec = tf.Variable(tf.zeros([1, 4*num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weighs and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))

  # Definition of cell computation
  def encoder_lstm_cell(i, o, state, dropout=False):
      if dropout:
          all_gates_state = tf.matmul(i, ifcox_enc) + tf.matmul(o, ifcom_enc) + ifcob_dec
      else:
          ifcom_dropout = tf.nn.dropout(ifcom_enc, keep_prob=0.9)
          all_gates_state = tf.matmul(i, ifcox_enc) + tf.matmul(o, ifcom_dropout) + ifcob_enc     
      input_gate = tf.sigmoid(all_gates_state[:, 0:num_nodes])
      forget_gate = tf.sigmoid(all_gates_state[:, num_nodes: 2*num_nodes])
      update = all_gates_state[:, 2*num_nodes: 3*num_nodes]
      state = forget_gate * state + input_gate * tf.tanh(update)
      output_gate = tf.sigmoid(all_gates_state[:, 3*num_nodes:])
      return output_gate * tf.tanh(state), state
    
  def decoder_lstm_cell(i, o, state, dropout=False):
      if dropout:
          all_gates_state = tf.matmul(i, ifcox_dec) + tf.matmul(o, ifcom_dec) + ifcob_dec
      else:
          ifcom_dropout = tf.nn.dropout(ifcom_dec, keep_prob=0.9)
          all_gates_state = tf.matmul(i, ifcox_dec) + tf.matmul(o, ifcom_dropout) + ifcob_dec
      input_gate = tf.sigmoid(all_gates_state[:, 0:num_nodes])
      forget_gate = tf.sigmoid(all_gates_state[:, num_nodes: 2*num_nodes])
      update = all_gates_state[:, 2*num_nodes: 3*num_nodes]
      state = forget_gate * state + input_gate * tf.tanh(update)
      output_gate = tf.sigmoid(all_gates_state[:, 3*num_nodes:])
      return output_gate * tf.tanh(state), state
    
  # Input data
  train_data = list()
  for _ in range(num_unrollings + 1):
      train_data.append(tf.placeholder(tf.float32, shape = [batch_size, vocabulary_size]))
  train_inputs = train_data

  train_labels = list()
  for _ in range(num_unrollings + 1): 
    train_labels.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))

  output = tf.constant(0.0, shape=[1,num_nodes], dtype=tf.float32)
  state = saved_state
    
  ## Unrolled encoder LSTM loop
  for i in train_inputs:
      output, state = encoder_lstm_cell(i, output, state, dropout=False)
    
  ## Last output from encoder is part of predition
  outputs = list([output])

  ## Unrolled encoder LSTM loop
  for i in train_labels:
      output, state = decoder_lstm_cell(i, output, state, dropout=False)
      outputs.append(output)


##  # State saving across unrollings
##  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
##      # Classifier            
##      logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
##      loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

  logits = tf.nn.xw_plus_b(tf.concat(0, outputs[:-1]), w, b)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    1.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  ## TODO: Generate a few validation sentences 
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[num_unrollings, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes], dtype=tf.float32))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes], dtype=tf.float32))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1.0, num_nodes])),
    saved_sample_state.assign(tf.zeros([1.0, num_nodes])))

  with tf.control_dependencies([saved_sample_output.assign(saved_sample_output),
                                saved_sample_state.assign(saved_sample_state)]):
    initial_prediction = tf.nn.softmax(tf.nn.xw_plus_b(saved_sample_output, w, b))
    for i in range(num_unrollings):
      saved_sample_output, saved_sample_state = encoder_lstm_cell(tf.reshape(sample_input[i,:], [1,vocabulary_size]), saved_sample_output, saved_sample_state)
      saved_sample_output, saved_sample_state = decoder_lstm_cell(initial_prediction, saved_sample_output, saved_sample_state)
      prediction = tf.nn.softmax(tf.nn.xw_plus_b(saved_sample_output, w, b))

In [19]:
num_steps = 8000
summary_frequency = 200

validation_sentence_1 = 'You spin my head right round, right round'
validation_sentence_2 = 'When you go down, when you go down down'
X_validation =  np.zeros(shape=(len(validation_sentence_1), vocabulary_size), dtype=np.float)
for b in range(len(validation_sentence_1)):
    X_validation[b, char2id(validation_sentence_1[b])] = 1.0
    
def prob_to_char(probs):
    return id2char(np.argmax(probs))
    
train_batches = BatchGenerator(batch_size, text, num_unrollings)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches, batches_labels = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i,:,:].T
      feed_dict[train_labels[i]] = batches_labels[i,:,:].T
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      #print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, batches_labels.reshape((1344,29))))))
      if step % (summary_frequency * 10) == 0:
        reset_sample_state.run()
        # Generate some samples.
        print('=' * 80)
        reset_sample_state.run()
        sentence = initial_prediction.eval()
        for _ in range(len(validation_sentence_1)):
          predictions = prediction.eval({sample_input: feed})
          feed = sample(prediction)
          sentence += characters(feed)[0]
        print(sentence)
        print('=' * 80)      
        # TODO: Measure validation set perplexity.

Unexpected character: Y
Unexpected character: ,
Initialized
Average loss at step 0: 3.364308 learning rate: 1.000000
 


NameError: name 'sample_prediction' is not defined

In [159]:
1344 / 64

21